# P.4 Optimizing with TensorBoard

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Activation, MaxPool2D
import pickle
import time

# tensorboard imports
from tensorflow.keras.callbacks import TensorBoard

In [2]:
# Define tensorboard log directory

NAME = 'Cat-vs-Dog-cnn-64X2-{}'.format(time.time())

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME), histogram_freq=1)

In [3]:
# load datasets from file

X = pickle.load(open('X.pickle', 'rb'))
y = pickle.load(open('y.pickle', 'rb'))

In [4]:
# normalize the data(feature datasets)

X = X/255.0

In [ ]:
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = '{}-conv-{}-node-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

In [6]:
# define model with various combinations of convolution and dense layers having different layer sizes

#dense_layers = [0, 1, 2]
#layer_sizes = [32, 64, 128]
#conv_layers = [1, 2, 3]

dense_layers = [1]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = '{}-conv-{}-node-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME), histogram_freq=1)
            model = Sequential()

            model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPool2D(pool_size=(2, 2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPool2D(pool_size=(2, 2)))
            
            model.add(Flatten()) # Flatten the data before applying to dense layers(vectorization of data)
            
            for l in range(dense_layer):
                #model.add(Dense(layer_size))
                model.add(Dense(512))
                model.add(Activation('relu'))

            model.add(Dense(1)) # final output layer and apply activation function since its a probability prediction
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
            model.summary()
            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
activation_108 (Activation)  (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 22, 22, 64)        36928     
_________________________________________________________________
activation_109 (Activation)  (None, 22, 22, 64)        0         
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 9, 9, 64)        

In [7]:
# Save the model

model.save('64x3-CNN.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: 64x3-CNN.model\assets


In [13]:
# perform prediction on trained model
import cv2

CATEGORIES = ['Cat', 'Dog']

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

model =tf.keras.models.load_model('64x3-CNN.model')

prediction = model.predict([prepare('keeshond_dog_pictures (1).jpg')])
print(CATEGORIES[int(prediction)])

Dog
